In [11]:
import string
from pathlib import Path
from collections import defaultdict
import json
import numpy as np
import math
file = open('./input.txt', 'r')

for line in file.readlines():
    line = line.replace('\n','')
    jets = [1 if l=='>' else -1 for l in line]
    jets2 = [l for l in line]
    print(f'readed jets line')

print(len(jets))
print(sum(jets))

readed jets line
10091
-159


In [12]:
rock_types = {
    0: [2, 5, np.array([0,0,0,0]), np.array([1,1,1,1])],
    1: [2, 4, np.array([1,0,1]), np.array([2,3,2])],
    2: [2, 4, np.array([0,0,0]), np.array([1,1,3])],
    3: [2, 2, np.array([0]), np.array([4])],
    4: [2, 3, np.array([0,0]), np.array([2,2])],
}
rock_shapes = {}
for idx, rock in rock_types.items():
    x1, x2, min_heights, heights = rock
    width = x2-x1+1
    height = max(heights)
    rock_shape = np.zeros((width,height))
    for i in range(width):
        rock_shape[i,min_heights[i]:heights[i]] = 1
    rock_shapes[idx] = rock_shape

print(rock_shapes)

{0: array([[1.],
       [1.],
       [1.],
       [1.]]), 1: array([[0., 1., 0.],
       [1., 1., 1.],
       [0., 1., 0.]]), 2: array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 1., 1.]]), 3: array([[1., 1., 1., 1.]]), 4: array([[1., 1.],
       [1., 1.]])}


In [13]:
cave = np.zeros((7,6000))
highest_point = 0
num_rocks_stopped = 0
rock_index = 0
is_stopped = True

def can_move(cave, move_x, rock_shape, x1, x2, y_pos):
    y1 = int(min(y_pos))
    old_window = cave[x1:x2+1, y1:y1+rock_shape.shape[1]]
    window = cave[x1+move_x:x2+move_x+1, y1:y1+rock_shape.shape[1]]
    assert window.shape == rock_shape.shape
    ans = not (np.bitwise_and(window>0, rock_shape>0).any())

    # if (np.bitwise_and(old_window>0, rock_shape>0).any()):
    #     print(f'rock is on another rock moving l/r??')
    #     visible_cave = cave.copy()
    #     visible_cave[x1:x2+1, min(y_pos):min(y_pos)+rock_shape.shape[1]] += 2*rock_shape
    #     print(f'with cave {visible_cave[:,:int(max(y_pos)+5)].T[::-1,:]}')
    #     print(f'with y_pos : {y_pos}')
    #     print(f'with x1, x2 : {x1},{x2}')
    #     print(f'with shape {rock_shape.T[::-1,:]}')
    #     print(f'window is {window.T[::-1,:]}')
    #     print(f'can move is {ans}')
    #     assert False

    return ans

def can_move_down(cave, rock_shape, x1, x2, y_pos):

    y1 = int(min(y_pos))
    if y1==0:
        return False
    old_window = cave[x1:x2+1, y1:y1+rock_shape.shape[1]]
    window = cave[x1:x2+1, y1-1:y1-1+rock_shape.shape[1]]
    assert window.shape == rock_shape.shape
    ans = not (np.bitwise_and(window>0, rock_shape>0).any())

    # if (np.bitwise_and(old_window>0, rock_shape>0).any()):
    #     print(f'rock is on another rock going down??')
    #     visible_cave = cave.copy()
    #     visible_cave[x1:x2+1, min(y_pos):min(y_pos)+rock_shape.shape[1]] += 2*rock_shape
    #     print(f'with cave {visible_cave[:,:int(max(y_pos)+5)].T[::-1,:]}')
    #     print(f'with old window : {old_window}')
    #     print(f'with y_pos : {y_pos}')
    #     print(f'with x1, x2 : {x1},{x2}')
    #     print(f'with shape {rock_shape.T[::-1,:]}')
    #     print(f'window is {window.T[::-1,:]}')
    #     print(f'can move is {ans}')
    #     assert False

    return ans

In [14]:
jet_idx = -1
caves_wrong = []
while num_rocks_stopped<2022:
    jet_idx = (jet_idx+1)%len(jets)
    if is_stopped:
        is_stopped = False
        current_rock = rock_types[rock_index].copy()
        rock_shape = rock_shapes[rock_index]
        x1, x2, min_heights, heights = current_rock
        y_pos = min_heights+highest_point+3

    move_x = jets[jet_idx]
    if move_x>0:
        if x2<6 and can_move(cave, move_x, rock_shape, x1, x2, y_pos):
            x1+=1
            x2+=1
    else:
        if x1>0 and can_move(cave, move_x, rock_shape, x1, x2, y_pos):
            x1+=-1
            x2+=-1
    
    movable = can_move_down(cave, rock_shape, x1, x2, y_pos)

    # visible_cave = cave.copy()
    # visible_cave[x1:x2+1, min(y_pos):min(y_pos)+rock_shape.shape[1]] = 2*rock_shape
    # print(f'movable down is {movable}')
    # print(f'highest point is {highest_point}')
    # print(f'y pos is {y_pos}')
    # print(f'CURRENT CAVE : \n {visible_cave[:,:highest_point+5].T[::-1,:]}')
    
    if movable:
        y_pos = y_pos-1
    else:
        bottom = min(y_pos)
        is_stopped = True
        rock_index = (rock_index+1)%5
        num_rocks_stopped += 1

        cave[x1:x2+1, bottom:bottom+rock_shape.shape[1]] += rock_shape
        xs, ys = np.where(cave>0)
        highest_point = max(ys)+1
        # print(f'cave after stopping : {cave[:,:highest_point+2].T[::-1,:]}')
        # print(f'highest point {highest_point}')
        # print(f'num moves = {jet_idx+1}')
        # print(f'stopping {num_rocks_stopped}th rock {rock_index%5} at h {bottom}')
        # caves_wrong.append(cave[:,:highest_point+2].copy())


In [15]:
print(highest_point)
print(num_rocks_stopped)
xs, ys = np.where(cave>0)
print(max(ys))

3181
2022
3180


In [16]:
def ext_imp_info(cave):
    xs, ys = np.where(cave>0)
    if len(ys)>0:
        ymax = int(max(ys))
        diffs = [0,0,0,0,0,0,0]
        for x in range(7):
            col = cave[x,:]
            col_blocked = np.where(col>0)[0].tolist()+[0]
            diffs[x] = int(min([ymax-y for y in col_blocked]))
        diffs = tuple(diffs)
    else:
        diffs = (0,0,0,0,0,0,0)
    return diffs

In [17]:
num_rocks_to_land = 1000000000000
cave = np.zeros((7,10000))
highest_point = 0
num_rocks_stopped = 0
num_rocks_stopped_remaining = 0
rock_index = 0
is_stopped = True
to_stop = False
started_remaining = False
jet_idx = -1
states = []
states_val = {}
h_by_rocks = {}

while not to_stop:

    jet_idx = (jet_idx+1)%len(jets)
    state = (int(jet_idx), int(rock_index), ext_imp_info(cave))
    states.append(state)
    h_by_rocks[num_rocks_stopped] = highest_point
    if not started_remaining and state in states_val.keys():
        recurrent_states = (states_val[state], (num_rocks_stopped, highest_point))
        started_remaining = True
        rock_cycle = num_rocks_stopped - states_val[state][0]
        height_cycle = highest_point - states_val[state][1]
        num_cycles = (num_rocks_to_land-num_rocks_stopped)//rock_cycle
        remaining_rocks = num_rocks_to_land - num_rocks_stopped - (num_cycles)*rock_cycle
        print(f'FOUND CYCLE, REMAINING {remaining_rocks} ROCKS')
        # break
    states_val[state] = (num_rocks_stopped, highest_point)

    if is_stopped:
        is_stopped = False
        current_rock = rock_types[rock_index].copy()
        rock_shape = rock_shapes[rock_index]
        x1, x2, min_heights, heights = current_rock
        y_pos = min_heights+highest_point+3

    move_x = jets[jet_idx]
    if move_x>0:
        if x2<6 and can_move(cave, move_x, rock_shape, x1, x2, y_pos):
            x1+=1
            x2+=1
    else:
        if x1>0 and can_move(cave, move_x, rock_shape, x1, x2, y_pos):
            x1+=-1
            x2+=-1
    
    movable = can_move_down(cave, rock_shape, x1, x2, y_pos)
    
    if movable:
        y_pos = y_pos-1
    else:
        bottom = min(y_pos)
        is_stopped = True
        rock_index = (rock_index+1)%5
        num_rocks_stopped += 1

        cave[x1:x2+1, bottom:bottom+rock_shape.shape[1]] += rock_shape
        xs, ys = np.where(cave>0)
        highest_point = max(ys)+1

        if started_remaining:
            num_rocks_stopped_remaining += 1
            if num_rocks_stopped_remaining>= remaining_rocks:
                to_stop=True


FOUND CYCLE, REMAINING 1398 ROCKS


In [18]:
print(highest_point)
print(num_rocks_stopped)
xs, ys = np.where(cave>0)
print(max(ys))

height_in_cycle = num_cycles*height_cycle
print(height_in_cycle+highest_point)

5247
3325
5246
1570434782634


In [21]:
print(recurrent_states)

n_rock1 = recurrent_states[0][0]
n_rock2 = recurrent_states[1][0]

h1 = recurrent_states[0][1]
h2 = recurrent_states[1][1]

assert h_by_rocks[n_rock1] == h1
assert h_by_rocks[n_rock2] == h2

rock_cycle = n_rock2-n_rock1
h_cycle = h2-h1

tot_h = h1
tot_rocks = n_rock1
num_cycles = (num_rocks_to_land-n_rock1)//rock_cycle

tot_h += h_cycle*num_cycles
tot_rocks += rock_cycle*num_cycles

remaining_rocks = num_rocks_to_land-tot_rocks
assert remaining_rocks<rock_cycle
remaining_h = h_by_rocks[n_rock1+remaining_rocks] - h_by_rocks[n_rock1]

tot_h += remaining_h
tot_rocks += remaining_rocks

print(tot_h)
print(tot_rocks)

((202, 329), (1927, 3038))
1570434782634
1000000000000
